In [4]:
import json
from datetime import datetime


def fixup_str(text):
    return text.encode('latin1').decode('utf8')


def fixup_list(l):
    return [fixup(e) for e in l]


def fixup_dict(dct):
    return {fixup_str(k): fixup(v) for k, v in dct.items()}


def fixup(e):
    if isinstance(e, dict):
        return fixup_dict(e)
    if isinstance(e, list):
        return fixup_list(e)
    if isinstance(e, str):
        return fixup_str(e)
    return e


def filter_messages(messages):
    for message in messages:
        if message['type'] == 'Generic':
            yield message


def format_message(message):
    sender = message['sender_name']
    timestamp = datetime.fromtimestamp(message['timestamp_ms']/1000)
    content = message.get('content', "")
    return timestamp, sender, content


def main():
    files = ['message_1.json', 'message_2.json', 'message_3.json', 'message_4.json', 'message_5.json', 'message_6.json']
    out = []

    with open('json_out.json', 'w') as json_msg:
        with open(files[0], 'r') as f:
            parts = fixup(json.loads(f.read()))['participants']
        for fname in files:
            with open(fname, 'r') as f:
                messages = fixup(json.loads(f.read()))['messages']

            for message in filter_messages(messages):
                #print(format_message(message))
                time, sender, msg = format_message(message)
                if msg == "":
                    continue
                d = {'time': str(time), 'sender': sender, 'message': msg}
                out.append(d.copy())
            out_msg = {'participants': parts, 'messages': out}
        json.dump(out_msg, json_msg, indent=4)


if __name__ == '__main__':
    main()